In [2]:
# Geological Storage ML Prediction - Jupyter Notebook
# This notebook implements ML models for predicting CO2 geological storage capacity

# %% [markdown]
# # Geological Storage Capacity Prediction using Machine Learning
# 
# ## Objective
# Predict CO2 storage capacity in geological formations based on:
# - Porosity
# - Permeability
# - Depth
# - Pressure
# - Temperature

# %% Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

# %% [markdown]
# ## 1. Data Generation
# 
# We'll generate synthetic data that simulates geological storage characteristics

# %% Generate Synthetic Data
def generate_geological_data(n_samples=500, random_state=42):
    """
    Generate synthetic geological storage data
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to generate
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    pd.DataFrame : Generated dataset
    """
    np.random.seed(random_state)
    
    # Generate features
    porosity = np.random.uniform(0.1, 0.4, n_samples)
    permeability = np.random.uniform(50, 250, n_samples)
    depth = np.random.uniform(500, 2500, n_samples)
    pressure = np.random.uniform(5, 25, n_samples)
    temperature = np.random.uniform(30, 100, n_samples)
    
    # Generate target variable with realistic relationships
    storage_capacity = (
        1000 * porosity * 0.5 +           # Porosity contributes positively
        permeability * 0.3 +               # Permeability contributes positively
        depth * 0.05 +                     # Depth contributes positively
        -pressure * 2 +                    # Pressure contributes negatively
        temperature * 0.5 +                # Temperature contributes positively
        np.random.normal(0, 20, n_samples) # Add noise
    )
    
    # Create DataFrame
    df = pd.DataFrame({
        'porosity': porosity,
        'permeability_md': permeability,
        'depth_m': depth,
        'pressure_mpa': pressure,
        'temperature_c': temperature,
        'storage_capacity_tons': storage_capacity
    })
    
    return df

# Generate data
df = generate_geological_data(n_samples=500)
print(f"✅ Generated {len(df)} samples")
print(f"\nDataset shape: {df.shape}")
print("\nFirst few rows:")
df.head()

# %% [markdown]
# ## 2. Exploratory Data Analysis (EDA)

# %% Basic Statistics
print("📊 Dataset Statistics:\n")
print(df.describe())

print("\n📊 Data Types:\n")
print(df.dtypes)

print("\n📊 Missing Values:\n")
print(df.isnull().sum())

# %% Visualize Distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Feature Distributions', fontsize=16, fontweight='bold')

for idx, col in enumerate(df.columns):
    ax = axes[idx // 3, idx % 3]
    ax.hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    ax.set_title(col.replace('_', ' ').title())
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% Correlation Analysis
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# %% Pairplot
print("Creating pairplot (this may take a moment)...")
sns.pairplot(df, diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pairwise Feature Relationships', y=1.02, fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 3. Data Preprocessing

# %% Split Data
# Separate features and target
X = df.drop('storage_capacity_tons', axis=1)
y = df['storage_capacity_tons']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ Data split complete!")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

# %% Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled using StandardScaler")

# %% [markdown]
# ## 4. Model Training

# %% Initialize Models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}

# %% Train and Evaluate Models
print("🚀 Training models...\n")

for name, model in models.items():
    print(f"Training {name}...")
    
    # Train model
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    if name == 'Linear Regression':
        cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                     cv=5, scoring='r2')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, 
                                     cv=5, scoring='r2')
    
    results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred_test': y_pred_test
    }
    
    print(f"✅ {name} trained successfully!")
    print(f"   Train R²: {train_r2:.4f}")
    print(f"   Test R²: {test_r2:.4f}")
    print(f"   Test RMSE: {test_rmse:.2f}")
    print(f"   CV Score: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})\n")

# %% [markdown]
# ## 5. Model Comparison

# %% Compare Models
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train R²': [results[m]['train_r2'] for m in results],
    'Test R²': [results[m]['test_r2'] for m in results],
    'Test RMSE': [results[m]['test_rmse'] for m in results],
    'Test MAE': [results[m]['test_mae'] for m in results],
    'CV Score': [results[m]['cv_mean'] for m in results]
})

print("📊 Model Comparison:\n")
print(comparison_df.to_string(index=False))

# %% Visualize Model Comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² Scores
axes[0].bar(comparison_df['Model'], comparison_df['Test R²'], 
            color=['#3498db', '#e74c3c', '#2ecc71'], alpha=0.7, edgecolor='black')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Model Performance (R² Score)', fontsize=14, fontweight='bold')
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3)

# RMSE
axes[1].bar(comparison_df['Model'], comparison_df['Test RMSE'], 
            color=['#3498db', '#e74c3c', '#2ecc71'], alpha=0.7, edgecolor='black')
axes[1].set_ylabel('RMSE', fontsize=12)
axes[1].set_title('Model Performance (RMSE)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Best Model Analysis

# %% Select Best Model
best_model_name = comparison_df.loc[comparison_df['Test R²'].idxmax(), 'Model']
best_model_results = results[best_model_name]

print(f"🏆 Best Model: {best_model_name}")
print(f"   Test R²: {best_model_results['test_r2']:.4f}")
print(f"   Test RMSE: {best_model_results['test_rmse']:.2f}")
print(f"   Test MAE: {best_model_results['test_mae']:.2f}")

# %% Actual vs Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, best_model_results['y_pred_test'], 
            alpha=0.6, edgecolor='black', s=50)
plt.plot([y_test.min(), y_test.max()], 
         [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Storage Capacity (tons)', fontsize=12)
plt.ylabel('Predicted Storage Capacity (tons)', fontsize=12)
plt.title(f'{best_model_name}: Actual vs Predicted', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# %% Residuals Analysis
residuals = y_test - best_model_results['y_pred_test']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals plot
axes[0].scatter(best_model_results['y_pred_test'], residuals, 
                alpha=0.6, edgecolor='black')
axes[0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0].set_xlabel('Predicted Values', fontsize=12)
axes[0].set_ylabel('Residuals', fontsize=12)
axes[0].set_title('Residuals Plot', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Residuals distribution
axes[1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residuals', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Residuals Distribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% Feature Importance (for tree-based models)
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': best_model_results['model'].feature_importances_
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'], 
             edgecolor='black', alpha=0.7)
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title(f'{best_model_name}: Feature Importance', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Feature Importance:\n")
    print(feature_importance.to_string(index=False))

# %% [markdown]
# ## 7. Make Predictions

# %% Prediction Function
def predict_storage_capacity(porosity, permeability, depth, pressure, temperature):
    """
    Predict CO2 storage capacity for given geological parameters
    
    Parameters:
    -----------
    porosity : float (0.1 - 0.4)
    permeability : float (50 - 250 mD)
    depth : float (500 - 2500 m)
    pressure : float (5 - 25 MPa)
    temperature : float (30 - 100 °C)
    
    Returns:
    --------
    float : Predicted storage capacity in tons
    """
    input_data = np.array([[porosity, permeability, depth, pressure, temperature]])
    
    if best_model_name == 'Linear Regression':
        input_scaled = scaler.transform(input_data)
        prediction = best_model_results['model'].predict(input_scaled)[0]
    else:
        prediction = best_model_results['model'].predict(input_data)[0]
    
    return prediction

# %% Example Predictions
print("🔮 Making sample predictions:\n")

test_cases = [
    {'porosity': 0.25, 'permeability': 150, 'depth': 1500, 'pressure': 15, 'temperature': 60},
    {'porosity': 0.35, 'permeability': 200, 'depth': 2000, 'pressure': 10, 'temperature': 80},
    {'porosity': 0.15, 'permeability': 100, 'depth': 1000, 'pressure': 20, 'temperature': 40}
]

for i, case in enumerate(test_cases, 1):
    prediction = predict_storage_capacity(**case)
    print(f"Test Case {i}:")
    print(f"  Porosity: {case['porosity']}, Permeability: {case['permeability']} mD")
    print(f"  Depth: {case['depth']} m, Pressure: {case['pressure']} MPa, Temp: {case['temperature']}°C")
    print(f"  ➜ Predicted Storage Capacity: {prediction:.2f} tons\n")

# %% [markdown]
# ## 8. Save Model and Results

# %% Save Model
import joblib

# Save the best model
model_filename = f'geological_storage_{best_model_name.lower().replace(" ", "_")}_model.pkl'
joblib.dump(best_model_results['model'], model_filename)
print(f"✅ Model saved as: {model_filename}")

# Save scaler (if Linear Regression)
if best_model_name == 'Linear Regression':
    joblib.dump(scaler, 'scaler.pkl')
    print("✅ Scaler saved as: scaler.pkl")

# %% Save Results
# Save comparison results
comparison_df.to_csv('model_comparison_results.csv', index=False)
print("✅ Results saved as: model_comparison_results.csv")

# Save predictions
predictions_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': best_model_results['y_pred_test'],
    'Residual': residuals.values
})
predictions_df.to_csv('test_predictions.csv', index=False)
print("✅ Test predictions saved as: test_predictions.csv")

# %% [markdown]
# ## 9. Summary Report

# %% Generate Report
print("=" * 60)
print("GEOLOGICAL STORAGE ML MODEL - FINAL REPORT")
print("=" * 60)
print(f"\n📊 Dataset Information:")
print(f"   Total samples: {len(df)}")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Features: {list(X.columns)}")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"\n📈 Performance Metrics:")
print(f"   Train R² Score: {best_model_results['train_r2']:.4f}")
print(f"   Test R² Score: {best_model_results['test_r2']:.4f}")
print(f"   Test RMSE: {best_model_results['test_rmse']:.2f} tons")
print(f"   Test MAE: {best_model_results['test_mae']:.2f} tons")
print(f"   Cross-Validation Score: {best_model_results['cv_mean']:.4f} (±{best_model_results['cv_std']:.4f})")

print(f"\n💾 Saved Files:")
print(f"   - {model_filename}")
if best_model_name == 'Linear Regression':
    print(f"   - scaler.pkl")
print(f"   - model_comparison_results.csv")
print(f"   - test_predictions.csv")

print("\n" + "=" * 60)
print("Analysis Complete! ✅")
print("=" * 60)

✅ Libraries imported successfully!
✅ Generated 500 samples

Dataset shape: (500, 6)

First few rows:
📊 Dataset Statistics:

         porosity  permeability_md      depth_m  pressure_mpa  temperature_c  \
count  500.000000       500.000000   500.000000    500.000000     500.000000   
mean     0.249569       146.390279  1535.116244     14.929530      64.989053   
std      0.089607        57.098691   594.385866      5.740197      20.006641   
min      0.101518        50.926405   509.879962      5.064365      30.109557   
25%      0.172384        95.819850   982.456097      9.821485      48.767853   
50%      0.253949       144.364313  1579.476672     15.177827      64.720873   
75%      0.326837       195.267364  2054.687516     19.747525      82.030536   
max      0.397889       249.943535  2498.827452     24.966950      99.680626   

       storage_capacity_tons  
count             500.000000  
mean              247.915257  
std                62.394160  
min                83.918877  
